In [92]:
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import requests

In [93]:
df = pd.read_csv("data/bangkok_traffy.csv")
df.describe()

,star,count_reopen
count,274097.000000,787026.000000
mean,3.872801,0.123485
std,1.471863,0.823554
min,1.000000,0.000000
25%,3.000000,0.000000
50%,5.000000,0.000000
75%,5.000000,0.000000
max,5.000000,100.000000


In [94]:
df.sample(5)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity
628678,2024-ARP2U6,{ถนน},"เขตบางกะปิ,สำนักการจราจรและขนส่ง กรุงเทพมหานคร...",ทางม้าลายเส้นจางหาย,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.62535,13.75469",26 ซอย แยก แขวง หัวหมาก เขตบางกะปิ กรุงเทพมหาน...,หัวหมาก,บางกะปิ,กรุงเทพมหานคร,2024-07-16 17:19:32.487926+00,เสร็จสิ้น,NaN,0,2024-12-18 18:02:01.611856+00
768068,2024-CFRMK6,"{ทางเท้า,ถนน}","เขตสาทร,สำนักการโยธา กทม.,สำนักงานก่อสร้างและบ...",การเรียงทางเท้า สีเข้ม บาง อ่อน ควรจะสลับให้เ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53226,13.71841",653 19 ระหว่าง ซอย นราธิวาสราชนครินทร์ 7 แขวงท...,ทุ่งมหาเมฆ,สาทร,กรุงเทพมหานคร,2024-12-23 03:20:26.93491+00,เสร็จสิ้น,NaN,0,2024-12-27 03:37:59.934947+00
621565,2024-9G3R9H,{},"เขตสายไหม,ฝ่ายสิ่งแวดล้อมฯ เขตสายไหม",มีสมาชิกผู้พักอาศัย ในหมู่บ้าน ได้รับผลกระทบป่...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.67303,13.89503",6 จตุโชติ 10 แขวงออเงิน เขตสายไหม กรุงเทพมหานค...,ออเงิน,สายไหม,กรุงเทพมหานคร,2024-07-09 13:44:13.355778+00,เสร็จสิ้น,5.0,0,2024-07-12 01:46:55.885489+00
165855,2022-32GEHU,{ทางเท้า},"เขตราชเทวี,สำนักการโยธา กทม.,ผอ.เขตราชเทวี (นา...",ทางเท้าไม่เรียบ เป็นหลุมเป็นบ่อ,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.52999,13.76486",20 ถนน โยธี แขวง ทุ่งพญาไท เขตราชเทวี กรุงเทพม...,ทุ่งพญาไท,ราชเทวี,กรุงเทพมหานคร,2022-10-28 09:51:40.523634+00,เสร็จสิ้น,1.0,0,2022-10-31 07:24:12.619826+00
101892,2022-BPCUZM,{จราจร},"เขตยานนาวา,สน.บางโพงพาง,สำนักงานตำรวจแห่งชาติ ...",รถสองแถวจอดแช่ ติดสี่แยกไฟแดง รถติดมาก,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53261,13.69555",432/1 ถ. สาธุประดิษฐ์ แขวง บางโพงพาง เขต ยานนา...,บางโพงพาง,ยานนาวา,กรุงเทพมหานคร,2022-07-28 10:29:42.100387+00,เสร็จสิ้น,1.0,0,2022-07-28 13:50:47.148351+00


In [95]:
finished_df = df[df["state"] == "เสร็จสิ้น"]

In [96]:
filtered_df = finished_df.dropna(
    axis=0, how="any", subset=["timestamp", "last_activity"]
)

In [97]:
def toDate(serie):
    return pd.to_datetime(serie, format="ISO8601").dt.tz_localize(None)


filtered_df["timestamp"] = toDate(filtered_df["timestamp"])
filtered_df["last_activity"] = toDate(filtered_df["last_activity"])

filtered_df["duration"] = (
    filtered_df["last_activity"] - filtered_df["timestamp"]
).dt.total_seconds() // 60
filtered_df.sample(5)

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,duration
570754,2024-63QTVT,"{ความสะอาด,น้ำท่วม}","เขตจอมทอง,ฝ่ายโยธา เขตจอมทอง,พนักงานกวาด เขตจอ...",ซอยวุฒากาศ 40 ช่วงที่มีฝนตกหนักน้ำจะท่วมและเข้...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.46739,13.70533",54/5 ถนน วุฒากาศ แขวงบางค้อ เขตจอมทอง กรุงเทพม...,บางค้อ,จอมทอง,กรุงเทพมหานคร,2024-05-15 19:53:11.574667,เสร็จสิ้น,NaN,0,2024-06-03 02:16:16.233301,26303.0
148465,2022-3GBXCD,"{ความสะอาด,คลอง,ต้นไม้}","เขตมีนบุรี,ฝ่ายรักษาความสะอาดฯ เขตมีนบุรี,ผอ.เ...",มีขยะจำนวนมากถูกทิ้งใต้ต้นไม้ บริเวณ คอสะพานคล...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.78355,13.81191",36/2 ถ. คุ้มเกล้า แขวง แสนแสบ มีนบุรี กรุงเทพม...,แสนแสบ,มีนบุรี,กรุงเทพมหานคร,2022-09-21 07:37:05.468007,เสร็จสิ้น,5.0,0,2022-10-25 07:36:46.104653,48959.0
22910,2022-DD3XP3,{},"เขตบางกอกใหญ่,ผอ.เขตบางกอกใหญ่ (นายเกียรติวิสุ...",นโยบายการเวนคืนที่ดินเพื่อเชื่อมซอยเพชรเกษม 16...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.46963,13.73037",281 ซ. จรัญสนิทวงศ์ 1 แขวง วัดท่าพระ เขตบางกอก...,วัดท่าพระ,บางกอกใหญ่,กรุงเทพมหานคร,2022-06-12 03:48:29.493791,เสร็จสิ้น,NaN,0,2022-11-15 08:00:32.082101,224892.0
591230,2024-G7TXK3,{ท่อระบายน้ำ},"เขตจตุจักร,ฝ่ายโยธา เขตจตุจักร,การประปานครหลวง...",ขุดท่อมาหลายอาทิตย์เเล้วไม่มีช่างมาทำ ช่างก่อส...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.55373,13.82500",16/29 ซอย วิภาวดีรังสิต 19 แขวงจตุจักร เขตจตุจ...,จตุจักร,จตุจักร,กรุงเทพมหานคร,2024-06-06 12:22:32.239157,เสร็จสิ้น,1.0,0,2024-06-23 09:03:57.993363,24281.0
707554,2024-DBMQNL,{ความสะอาด},"เขตบางซื่อ,ฝ่ายรักษาความสะอาดฯ เขตบางซื่อ",มีคนมาทิ้งขยะหน้าบ้านส่งกลิ่นเหม็นรบกวน,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53739,13.82379",16 ถ. ประชาชื่น แขวงวงศ์สว่าง เขตบางซื่อ กรุงเ...,วงศ์สว่าง,บางซื่อ,กรุงเทพมหานคร,2024-10-13 06:06:24.083354,เสร็จสิ้น,NaN,0,2024-10-15 05:52:46.511569,2866.0


In [98]:
filtered_df["type"].fillna("{}", inplace=True)
filtered_df["type"].isna().sum()

/var/folders/06/w6kt9p1529vdfxw92dxt_6pc0000gn/T/ipykernel_22449/4244501731.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df['type'].fillna('{}', inplace=True)


np.int64(0)

In [99]:
types = {}


def count_type(row):
    val = row["type"]
    print(val)
    for t in val[1:-1].split(","):
        types[t] += 1

In [100]:
for data in filtered_df["type"]:
    # print(data)
    for t in data[1:-1].split(","):
        if t not in types:
            types[t] = 0
        types[t] += 1
# for key in sorted(types, reverse=True, key=lambda x: types[x]):
#     print(f'{key} - {types[key]}')
print(set(list(types)))

{'', 'ป้ายจราจร', 'สะพาน', 'PM2.5', 'กีดขวาง', 'คลอง', 'ห้องน้ำ', 'ป้าย', 'น้ำท่วม', 'จราจร', 'สัตว์จรจัด', 'ถนน', 'ท่อระบายน้ำ', 'เสียงรบกวน', 'สายไฟ', 'แสงสว่าง', 'ต้นไม้', 'ทางเท้า', 'เสนอแนะ', 'การเดินทาง', 'ความปลอดภัย', 'ร้องเรียน', 'สอบถาม', 'คนจรจัด', 'ความสะอาด'}


In [101]:
# columns = ['type', 'organization', 'coords', 'timestamp', 'duration']

# main_df = filtered_df[columns]

In [102]:
# Define allowed categories
target_types = {
    "",
    "ป้าย",
    "ความสะอาด",
    "แสงสว่าง",
    "สอบถาม",
    "ร้องเรียน",
    "การเดินทาง",
    "จราจร",
    "ท่อระบายน้ำ",
    "สะพาน",
    "เสียงรบกวน",
    "ต้นไม้",
    "คนจรจัด",
    "คลอง",
    "ถนน",
    "เสนอแนะ",
    "กีดขวาง",
    "สายไฟ",
    "PM2.5",
    "น้ำท่วม",
    "ทางเท้า",
    "สัตว์จรจัด",
    "ความปลอดภัย",
    "ห้องน้ำ",
    "ป้ายจราจร",
}


# Function to convert string to list
def parse_categories(s):
    # Remove curly braces and split
    return s.strip("{}").split(",")


# Parse the string column
filtered_df["type_list"] = filtered_df["type"].apply(parse_categories)

# Create binary flags
for cat in target_types:
    filtered_df[cat] = filtered_df["type_list"].apply(lambda lst: int(cat in lst))

# Create 'Others' column
filtered_df["Others"] = filtered_df["type_list"].apply(
    lambda lst: int(any(c not in target_types for c in lst))
)

# Drop intermediate column if you want
filtered_df.drop(columns="type_list", inplace=True)

filtered_df[["type"] + list(target_types)].sample(5)

,type,,สัตว์จรจัด,ป้ายจราจร,ถนน,สะพาน,ท่อระบายน้ำ,เสียงรบกวน,PM2.5,สายไฟ,...,เสนอแนะ,การเดินทาง,สอบถาม,ร้องเรียน,ความปลอดภัย,คนจรจัด,ป้าย,น้ำท่วม,จราจร,ความสะอาด
132949,{ถนน},0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118075,"{คลอง,ความสะอาด}",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
683386,{แสงสว่าง},0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
664044,"{PM2.5,เสียงรบกวน}",0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
210796,{ทางเท้า},0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
# Define working hours
start_time = time(9, 30)
end_time = time(15, 30)


def minutes_to_next_working_hour(dt):
    weekday = dt.weekday()  # 0 = Monday, ..., 6 = Sunday
    current_time = dt.time()

    # If it's a weekend or after working hours, find next weekday at 9:30
    if weekday >= 5 or current_time >= end_time:
        # Add days until next weekday (Monday–Friday)
        days_ahead = 1
        while (dt + timedelta(days=days_ahead)).weekday() >= 5:
            days_ahead += 1
        next_working_start = datetime.combine(
            (dt + timedelta(days=days_ahead)).date(), start_time
        )
        return int((next_working_start - dt).total_seconds() // 60)

    # Before work hours today
    elif current_time < start_time:
        today_start = datetime.combine(dt.date(), start_time)
        return int((today_start - dt).total_seconds() // 60)

    # During working hours
    else:
        return 0


# Apply function to timestamp column
filtered_df["until_working_time"] = filtered_df["timestamp"].apply(
    minutes_to_next_working_hour
)

filtered_df[["timestamp", "until_working_time"]].sample(5)

,timestamp,until_working_time
215914,2023-02-11 06:02:57.459617,3087
166587,2022-10-30 04:49:54.369230,1720
491662,2024-01-31 15:23:13.818305,0
482221,2024-01-21 02:00:06.092628,1889
484018,2024-01-23 03:43:30.909349,346


# Org's info

In [104]:
limit = 100
org_key = "bangkok"
url = f"https://publicapi.traffy.in.th/premium-org-fondue/statistic-rank/top-rank-avg-star?org_key={org_key}&limit={limit}"

resp = requests.get(url)
data = resp.json()
orgs = data["results"]

pd.DataFrame(orgs).to_csv("data/org.csv")

In [105]:
orgs = pd.read_csv("data/org.csv")
orgs.sample(5)
orgs.drop(columns=["Unnamed: 0"], inplace=True)
# orgs.reset_index(inplace=True)
orgs.set_index("fonduegroup_name", inplace=True)
orgs.sample(5)

,rank,timestamp,fonduegroup_id,post_count_star,post_count_star_stat,avg_star,post_count_all,post_count_finish,post_finish_percentage,avg_duration_minutes_inprogress,avg_duration_minutes_finished
fonduegroup_name,,,,,,,,,,,
เขตลาดกระบัง,54,NaN,18043,8025,"{'star_1': 1432, 'star_2': 403, 'star_3': 826,...",3.75,23586,19024,80.66,14266,84368
เขตหนองแขม,26,NaN,18052,3758,"{'star_1': 573, 'star_2': 147, 'star_3': 295, ...",3.96,12372,11236,90.82,8480,57869
เขตคลองสามวา,30,NaN,18057,6301,"{'star_1': 812, 'star_2': 304, 'star_3': 569, ...",3.93,17977,15310,85.16,18053,105648
เขตสัมพันธวงศ์,46,NaN,18049,2219,"{'star_1': 413, 'star_2': 110, 'star_3': 183, ...",3.80,6060,5052,83.37,6778,45202
สำนักสิ่งแวดล้อม กทม.,57,NaN,18129,9173,"{'star_1': 1749, 'star_2': 497, 'star_3': 885,...",3.72,24309,21640,89.02,8405,78992


In [106]:
# Define target columns
target_cols = ["avg_star", "post_finish_percentage", "avg_duration_minutes_finished"]

# Convert organization string to list
filtered_df["orgs_list"] = filtered_df["organization"].str.strip().str.split(",")

# Create filtered_df with NaNs in target columns
filtered_df[target_cols] = np.nan

# Explode orgs_list to merge with orgs
exploded = filtered_df[["orgs_list"]].explode("orgs_list").reset_index()
exploded["orgs_list"] = exploded["orgs_list"].str.strip()

# Join with orgs DataFrame
merged = exploded.merge(
    orgs[target_cols], left_on="orgs_list", right_index=True, how="left"
)

# Aggregate mean values for each original row
means = merged.groupby("index")[target_cols].mean()

# Assign the means back to filtered_df
filtered_df.loc[means.index, target_cols] = means

# Result
filtered_df[["orgs_list"] + target_cols]

,orgs_list,avg_star,post_finish_percentage,avg_duration_minutes_finished
0,[เขตบางซื่อ],4.08,84.940,85415.0
1,"[เขตประเวศ, ฝ่ายโยธา เขตประเวศ]",3.82,84.320,76399.0
2,[เขตสาทร],3.86,84.510,54852.0
3,"[เขตบางซื่อ, ฝ่ายโยธา เขตบางซื่อ]",4.08,84.940,85415.0
4,"[เขตลาดพร้าว, ฝ่ายโยธา เขตลาดพร้าว]",4.09,85.420,50067.0
...,...,...,...,...
786856,"[เขตพญาไท, สำนักการจราจรและขนส่ง กรุงเทพมหานคร...",3.92,73.165,138599.0
786892,"[เขตบางพลัด, สน.บวรมงคล]",3.59,72.540,92196.0
786893,"[เขตบางเขน, สน.บางเขน, สำนักงานตำรวจแห่งชาติ (...",4.00,86.020,65512.0
786944,"[เขตบางกอกน้อย, ฝ่ายเทศกิจ เขตบางกอกน้อย]",3.87,84.180,72567.0


In [107]:
filtered_df.to_csv("data/clean.csv")

In [108]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 645700 entries, 0 to 786956
Data columns (total 48 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   ticket_id                      638771 non-null  object        
 1   type                           645700 non-null  object        
 2   organization                   645700 non-null  object        
 3   comment                        638771 non-null  object        
 4   photo                          645605 non-null  object        
 5   photo_after                    620406 non-null  object        
 6   coords                         645700 non-null  object        
 7   address                        638771 non-null  object        
 8   subdistrict                    645355 non-null  object        
 9   district                       645353 non-null  object        
 10  province                       645652 non-null  object        
 11  times

In [109]:
rain_df = pd.read_csv("data/rain_data.csv")
filtered_df["date"] = filtered_df["timestamp"].dt.date.astype(str)
rain_df.head(1)

,date,rain_sum,precipitation_hours,precipitation_sum,is_rain
0,2021-07-31,3.2,6.0,3.2,True


In [110]:
merged_df = filtered_df.merge(
    rain_df[["date", "is_rain"]], left_on="date", right_on="date", how="left"
)

In [111]:
del merged_df["date"]
merged_df.to_csv("data/merged_data.csv")